In [3]:
import numpy as np
import os
from skimage.feature import local_binary_pattern
from skimage import io
from skimage.color import rgb2gray
from skimage.transform import resize
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam


In [4]:


# Dataset directories
train_path = os.path.join('Dataset/train')
test_path = os.path.join('Dataset/test')

# Step 2: Preprocess the Data
img_width, img_height = 48, 48
batch_size = 32



In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Use 20% of the images for validation
)

test_datagen = ImageDataGenerator(rescale=1./255)



In [6]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)



Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [7]:
# Step 3: Build the CNN Model
model = Sequential([
    Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices)),
    Activation('softmax')
])



In [8]:
# Step 4: Compile the Model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



In [9]:
# Step 5: Train the Model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)



Epoch 1/25
717/717 [==============================] - 229s 245ms/step - loss: 1.8016 - accuracy: 0.2572 - val_loss: 1.7115 - val_accuracy: 0.2970
Epoch 2/25
717/717 [==============================] - 28s 39ms/step - loss: 1.6847 - accuracy: 0.3203 - val_loss: 1.6074 - val_accuracy: 0.3623
Epoch 3/25
717/717 [==============================] - 28s 39ms/step - loss: 1.6041 - accuracy: 0.3664 - val_loss: 1.5136 - val_accuracy: 0.41383 - accuracy: 0. - ETA: 0s - loss: 1
Epoch 4/25
717/717 [==============================] - 29s 41ms/step - loss: 1.5332 - accuracy: 0.3983 - val_loss: 1.4466 - val_accuracy: 0.4469
Epoch 5/25
717/717 [==============================] - 34s 47ms/step - loss: 1.4926 - accuracy: 0.4250 - val_loss: 1.3954 - val_accuracy: 0.4693
Epoch 6/25
717/717 [==============================] - 33s 46ms/step - loss: 1.4481 - accuracy: 0.4412 - val_loss: 1.3730 - val_accuracy: 0.4766
Epoch 7/25
717/717 [==============================] - 31s 43ms/step - loss: 1.4246 - accuracy: 0.4

In [10]:
# Step 6: Evaluate the Model
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test accuracy: {test_acc}, Test loss: {test_loss}')

224/224 [==============================] - 38s 172ms/step - loss: 1.1705 - accuracy: 0.5571
Test accuracy: 0.5570591688156128, Test loss: 1.1705397367477417
